In [5]:
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine



In [4]:
df = pd.read_csv("spotify.csv", encoding='ISO-8859-1')
print(df.shape)
print(df.columns.tolist())

(953, 24)
['track_name', 'artist(s)_name', 'artist_count', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'in_spotify_charts', 'streams', 'in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts', 'bpm', 'key', 'mode', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%', 'instrumentalness_%', 'liveness_%', 'speechiness_%']


In [6]:
engine = create_engine("sqlite:///spotify.db")
df.to_sql("spotify", engine, index=False)

953

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM spotify WHERE Age < 2;")